In [1]:
def listDirs(newMMARDir):
    !ls $newMMARDir
    !echo ----config
    !ls $newMMARDir/config
    !echo ----commands
    !ls $newMMARDir/commands
def printFile(filePath,lnSt,lnOffset):
    print ("showing ",str(lnOffset)," lines from file ",filePath, "starting at line",str(lnSt))
    lnOffset=lnSt+lnOffset
    !< $filePath head -n "$lnOffset" | tail -n +"$lnSt"   

In [11]:
import IPython
IPython.display.IFrame('./provisioning_ui.html',width=850,height=700)

### 2 Run Provisioning tool 
For simplicity we have included a project1.yaml and project1auth.json files for you to use in this notebook.
In order to see their content simply run cell below 

In [2]:
!pwd

/mount/src/clara/Clara_hands-on/NVFlare/with_provision


In [3]:
MMAR_ROOT="/mount/src/clara/Clara_hands-on/NVFlare/with_provision/"
PROV_DIR="provisioning"
PROJ_NAME="test"

printFile(MMAR_ROOT+PROJ_NAME+".yml",0,50)

showing  50  lines from file  /mount/src/clara/Clara_hands-on/NVFlare/with_provision/test.yml starting at line 0
api_version: 1

# org is to describe each participant's organization
# schema change: org is now mandatory

# the name of this project
name: test

# homomorphic encryption
he_config:
  poly_modulus_degree: 8192
  coeff_mod_bit_sizes: [60, 40, 40]
  scale_bits: 40
  scheme: CKKS

config_folder: config

# Server enforcing role-based rights on admin users. true means all admin users are in role super
disable_authz: false

server:
  org: nvidia

  # set cn to the server's fully qualified domain name
  cn: localhost

  # replace the number with that all clients can reach out to, and that the server can open to listen to
  fed_learn_port: 8002

  # again, replace the number with that all clients can reach out to, and that the server can open to listen to
  # the value must be different from fed_learn_port
  admin_port: 8003

  # admin_storage is the mmar upload folder name on the 


### 2.1 Run provisioning tool
Cell below show help on how to use the cli for the provisioning tool 

In [4]:
!provision -h

usage: provision [-h] [-p PROJECT_FILE] [-o OUTPUT_DIR] [-n] [-u]

optional arguments:
  -h, --help            show this help message and exit
  -p PROJECT_FILE, --project_file PROJECT_FILE
                        file to describe FL project
  -o OUTPUT_DIR, --output_dir OUTPUT_DIR
                        directory to store all packages
  -n, --no_zip_password
                        Disable password protection in final zip files
  -u, --ui_tool         Run provisioning UI tool to generate project.yml file


In [5]:
%cd $MMAR_ROOT
!rm -r $PROJ_NAME
%mkdir -p $PROJ_NAME/$PROV_DIR

/mount/src/clara/Clara_hands-on/NVFlare/with_provision


In [6]:
PROJ_PATH=MMAR_ROOT+PROJ_NAME+"/"
PROV_PATH=PROJ_PATH+PROV_DIR+"/"
%cd $PROJ_PATH
!provision -p $MMAR_ROOT/$PROJ_NAME'.yml' -o $PROV_DIR

/mount/src/clara/Clara_hands-on/NVFlare/with_provision/test
Project yaml file: /mount/src/clara/Clara_hands-on/NVFlare/with_provision//test.yml.
Env var NVFL_DOCKER_IMAGE not set.  No docker.sh will be generated.
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ participant           ┃ org    ┃ destination              ┃ password         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ localhost             │ nvidia │ server.zip               │ SBg20dnNlfoAt8sI │
│ org1                  │ org1   │ org1.zip                 │ nK423Ehs8pfIdY15 │
│ nvidia                │ nvidia │ nvidia.zip               │ BJ0QqC2wOritzWoh │
│ admin@nvidia.com      │ nvidia │ admin@nvidia.com.zip     │ VXAzDgx86Ry1N2h9 │
│ researcher@nvidia.com │ nvidia │ researcher@nvidia.com.z… │ FplEYgyuk8W7JqPH │
│ researcher@org1.com   │ org1   │ researcher@org1.com.zip  │ A9XlEZQ0h1WLPD3U │
└───────────────────────┴────────┴────────────────────────

### 3. Send startup kits to participants
In a real experiment, you would send packages to each site so they would run it on their system. 
Here we would extract and simulate a server, 3 clients and an admin all in this tutorial. 

Cell above should have printed out passwords for each package. 
You should replace the password from above cell to the corresponding file in cell below 

In [7]:
%cd  $PROV_PATH
!unzip -oP SBg20dnNlfoAt8sI server.zip -d ../server
!unzip -oP nK423Ehs8pfIdY15 org1.zip -d ../org1
!unzip -oP VXAzDgx86Ry1N2h9 admin@nvidia.com.zip -d ../admin

/mount/src/clara/Clara_hands-on/NVFlare/with_provision/test/provisioning
Archive:  server.zip
   creating: ../server/startup/
  inflating: ../server/startup/signature.pkl  
  inflating: ../server/startup/start.sh  
  inflating: ../server/startup/log.config  
  inflating: ../server/startup/server.key  
  inflating: ../server/startup/rootCA.pem  
  inflating: ../server/startup/sub_start.sh  
  inflating: ../server/startup/stop_fl.sh  
  inflating: ../server/startup/authorization.json  
  inflating: ../server/startup/server_context.tenseal  
  inflating: ../server/startup/server.crt  
  inflating: ../server/startup/fed_server.json  
  inflating: ../server/startup/readme.txt  
Archive:  org1.zip
   creating: ../org1/startup/
  inflating: ../org1/startup/signature.pkl  
  inflating: ../org1/startup/fed_client.json  
  inflating: ../org1/startup/start.sh  
  inflating: ../org1/startup/client_context.tenseal  
  inflating: ../org1/startup/log.config  
  inflating: ../org1/startup/client.key  

### 4. Double check on rights before sending kits to clients

As you may notice creating users and rights could be a bit tricky with large number of clients (20+).
Specially if each client has different requirements. 
In order to make this task a bit easier, 
we have created a checking tool where you pass the authorization.json, 
then check that it is doing what you intended to do.

In order to validate this you can use the shell tool as 
```
cd /claraDevDay/FL/project1/server/startup
authz_preview -p authorization.json
```  

This would open an interactive prompt. You can type `?` to see list of commands as 

```
>?

Documented commands (type help <topic>):
========================================
bye  help

Undocumented commands:
======================
eval_right  eval_user    show_rights  show_site_rules  show_users
eval_rule   show_config  show_rules   show_sites     
```


You can use `show_users` to find all users as 
```
------------------------------------------------------
| USER                    | ORG    | ROLES           |
------------------------------------------------------
| admin@admin.com         | nvidia | super           |
| leadIT@org1.com         | org1   | lead_researcher |
| siteresearcher@org2.com | org2   | site_researcher |
| leadIT@secure.com       | Secure | only4secure     |
------------------------------------------------------
```

Then in order to see rules for a certain site you can type <br>
`eval_user <user> <org name>`
for example: <br> 
`eval_user admin@admin.com server`
you should see something similar to 
```
-------------------------
| RIGHT        | RESULT |
-------------------------
| upload_mmar  | True   |
| deploy_self  | False  |
| deploy_all   | False  |
| train_self   | False  |
| train_all    | False  |
| view_self    | False  |
| view_all     | False  |
| operate_all  | False  |
| operate_self | False  |
-------------------------
```

In our sample client4 doens't allow anyone to upload any mmars. You can check by 
`eval_user leadIT@org1.com client4`
vs `eval_user leadIT@org1.com client1`

but user `leadIT@secure.com` can upload to client4. You can check by 
`eval_user leadIT@secure.com client4`

You could also verify only one specific right using 
```
eval_right leadIT@org1.com deploy_self client4
```


# Next steps 
1. You should run the [admin notebook](Admin.ipynb) to start your FL experiment.
2. You may want to take a look at the [client notebook](Client.ipynb)



# Exercise 

1. Provision your own FL experiment. For this you should 
    1. got back to cell2 and which displays the FLprovUI.html page 
    2. Make changes modify it with your client names and emails
    2. Get your server host name using any of the commands below 
    ```
    hostname
    hostnamectl
    cat /proc/sys/kernel/hostname
    ```
    3. In the yml file change the cn name to the server name as 
    ```
    server:
      cn: <your server name >
    ``` 
    4. Click generate artifacts 
    5. Save `provision.yml` and `authz_config.json` for your new project 
    6. Continue steps in the notebook 